In [1]:
import numpy as np

In [2]:
from crewai import Agent


def create_keywords_search_agent(llm):
    return Agent(
        name="keywords_search_agent",
        role="Search Query Strategist",
        goal="\n".join([
            "Generate a diverse set of highly relevant search queries.",
            "Ensure the queries target specific products or categories to maximize discovery."
        ]),
        backstory=(
            "An expert in consumer behavior and search engine strategies, this agent crafts "
            "targeted and creative search queries to improve product visibility and retrieval "
            "from online sources."
        ),
        llm=llm,
        verbose=True
    )

In [3]:
# tasks/keywords_search_task.py

from crewai import Task
from typing import List
from pydantic import BaseModel, Field
import os


def create_keywords_search_task(
    product_name: str,
    websites_list: str,
    country_name: str,
    no_keywords: int,
    language: str,
    search_queries_recommendation_agent
):
    class DynamicSuggestedSearchQueries(BaseModel):
        queries: List[str] = Field(
            ..., 
            title="Suggested search queries to be passed to the search engine",
            min_items=1,
            max_items=no_keywords
        )

    return Task(
        description="\n".join([
            f"Rankyx aims to find **{product_name}** with a focus on value-for-money options.",
            f"Search should target the following websites: {websites_list}.",
            f"Locate **all relevant product listings** available online to enable comprehensive price and feature comparison.",
            f"Only include products that are sold in **{country_name}**.",
            f"Generate up to **{no_keywords}** highly specific search queries.",
            f"Queries must be composed in **{language}**.",
            "Include specific **brands, product types, or technologies**  avoid vague or generic terms.",
            "Make sure queries lead to **e-commerce product pages** directly, excluding blogs, news articles, or category hubs."
        ]),
        expected_output="A JSON object containing a list of relevant and highly specific search queries.",
        output_json=DynamicSuggestedSearchQueries,
        output_file=os.path.join(os.path.abspath('../Data'), "step_1_suggested_search_queries.json"),
        agent=search_queries_recommendation_agent
    )


In [15]:
from Agents.keywords_Search_Agent import create_keywords_search_agent
from Tasks.Create_Keywords_search_Task import create_keywords_search_task   
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource  
from crewai import Crew, Process, LLM
import agentops
import os
import sys
from dotenv import load_dotenv



# Load API keys from .env file
load_dotenv()

DEEPSEEK_API_KEY = os.getenv("DEEP_SEEK_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
AGENT_OPS_KEY = os.getenv("AGENTOPS_API_KEY")
HF_TOKEN = os.getenv("HF_KEY")    

# Initialize AgentOps
agentops.init(
    api_key=AGENT_OPS_KEY,
    skip_auto_end_session=True,
    default_tags=['crewai']
)

# Use LLM class from CrewAI, not openai.OpenAI
DeepSeek_llm = LLM(
    api_key=HF_TOKEN,
    model="huggingface/DeepSeek-R1-0528",
    temperature=0.0,
    max_tokens=100
)

# Mistral_llm = LLM(
#     api_key=MISTRAL_API_KEY,
#     base_url="https://openrouter.ai/api/v1",
#     model="openrouter.ai/mistralai/devstral-small:free",
#     temperature=0.0,
#     max_tokens=100
# )

In [16]:
about_company = "Rankyx is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

# Create agent
keywords_search_agent = create_keywords_search_agent(DeepSeek_llm)

# Create task
keywords_search_task = create_keywords_search_task(
    product_name="iphone 16",
    websites_list=["www.amazon.eg", "www.jumia.com.eg"],
    country_name="Egypt",
    no_keywords=5,
    language="english",
    search_queries_recommendation_agent=keywords_search_agent
)

# Define the crew
rankyx_crew = Crew(
    agents=[keywords_search_agent],
    tasks=[keywords_search_task],
    process=Process.sequential,
    knowledge_sources=[company_context]
)

# Execute the crew
crew_results = rankyx_crew.kickoff(
    
)

# Print results
print(crew_results)


# Agent: Search Query Strategist
## Task: Rankyx aims to find **iphone 16** with a focus on value-for-money options.
Search should target the following websites: ['www.amazon.eg', 'www.jumia.com.eg'].
Locate **all relevant product listings** available online to enable comprehensive price and feature comparison.
Only include products that are sold in **Egypt**.
Generate up to **5** highly specific search queries.
Queries must be composed in **english**.
Include specific **brands, product types, or technologies**  avoid vague or generic terms.
Make sure queries lead to **e-commerce product pages** directly, excluding blogs, news articles, or category hubs.








 An unknown error occurred. Please check the details below.



APIConnectionError: litellm.APIConnectionError: not enough values to unpack (expected 2, got 1)
Traceback (most recent call last):
  File "c:\Users\aliay\OneDrive\Desktop\iTi\30 - GenAI\Smart-Shopper-ai\env\Lib\site-packages\litellm\main.py", line 2220, in completion
    response = base_llm_http_handler.completion(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aliay\OneDrive\Desktop\iTi\30 - GenAI\Smart-Shopper-ai\env\Lib\site-packages\litellm\llms\custom_httpx\llm_http_handler.py", line 261, in completion
    api_base = provider_config.get_complete_url(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\aliay\OneDrive\Desktop\iTi\30 - GenAI\Smart-Shopper-ai\env\Lib\site-packages\litellm\llms\huggingface\chat\transformation.py", line 95, in get_complete_url
    first_part, remaining = model.split("/", 1)
    ^^^^^^^^^^^^^^^^^^^^^
ValueError: not enough values to unpack (expected 2, got 1)
